In [ ]:
import gradio as gr
import cv2

from cnn2snn import set_akida_version, AkidaVersion
import akida

import numpy as np

import time

import plotly.graph_objects as go

In [ ]:
def create_gauge(value):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        gauge={'axis': {'range': [0, 30]}},
        domain={'x': [0, 1], 'y': [0, 1]},
    ))
    fig.update_layout(width=400, height=300)
    return fig

In [ ]:
# Softmax for an array of values
def softmaxArray(values):
    # Assuming array shape is (1, 1, 1, x), flatten to get the values
    values = values.ravel()
    exp_values = np.exp(values)
    sum_exp = np.sum(exp_values)
    softmax_values = exp_values / sum_exp
    return softmax_values

In [ ]:
image_x = 96
image_y = 96
image_z = 3
labels = ["no person", "person"]
def decodeOutput(inp):
        global akida_model
        inp = cv2.resize(inp, (image_x, image_y))
        inp = inp.reshape((-1, image_x, image_y, image_z))
        timer_start = time.time()
        predictions = softmaxArray(akida_model.predict(inp))
        frame_time = time.time() - timer_start
        fps = 1 / frame_time if frame_time > 0 else 0
        confidences = {labels[i]: predictions[i] for i in range(len(predictions))}

        return confidences, fps

In [ ]:
def classify_image(inp):

  confidences, fps = decodeOutput(inp)

  return confidences, create_gauge(round(fps, 2))

In [ ]:
from quantizeml import load_model
from akida_models import fetch_file

# Load the pre-trained quantized model
model_file = fetch_file(
    fname="akidanet_vww_i8_w4_a4.h5",
    origin="https://data.brainchip.com/models/AkidaV2/akidanet/akidanet_vww_i8_w4_a4.h5",
    cache_subdir='models')
model_keras_quantized = load_model(model_file)
model_keras_quantized.summary()


In [ ]:
from cnn2snn import convert

# Convert the model
akida_model = convert(model_keras_quantized)
akida_model.summary()

In [ ]:
with set_akida_version(AkidaVersion.v2):
            devices = akida.devices()
            if len(devices) > 0:
                print(f'Available devices: {[dev.desc for dev in devices]}')
                device = devices[0]
                print(device.version)
                try:
                    akida_model.map(device)
                    print(f"Mapping to Akida device {device.desc}.")
                    mappedDevice = device.version
                except Exception as e:
                    print("Model not compatible with FPGA. Running on CPU.")
                    mappedDevice = "CPU"
            else:
                print("No Akida devices found, running on CPU.")
                mappedDevice = "CPU"

In [ ]:
theme = gr.themes.Base(
    text_size="sm",
    spacing_size="sm",
    radius_size="sm",
)

with gr.Blocks(
    title="Brainchip",
    fill_width=True,
    fill_height=True,
    delete_cache=[180, 600],
    theme=theme
) as demo:
    gr.Markdown("""
        <h1 style="text-align: center;">Akida Cloud</h1>
        <br>
        """)
    with gr.Row():
        gr.Markdown("## Image Classification")
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(sources=["webcam"], type="numpy")
        with gr.Column():
            gr.Markdown(f"""Device: {mappedDevice}""")
            output_label = gr.Label(num_top_classes=3)
            plot = gr.Plot(label="Frames per second")
        dep = input_img.stream(classify_image, [input_img], [output_label, plot],
                                time_limit=30, stream_every=0.1, concurrency_limit=30)        


if __name__ == "__main__":
    demo.launch()